# Stats

This notebook loads some data, reports some simple descriptive statistics (means, standard deviationsm etc) and show a number of useful plots (scatter plots, histograms, time series plots).

Most of the descriptive stats use built-in Julia commands. The plots reply on the Plots package and the pdf and quantiles are from the Distributions package (see https://github.com/JuliaStats/Distributions.jl).

## Load Packages

In [1]:
using Distributions      #distributions, random numbers, etc.
                         
include("printmat.jl")   #just a function for prettier matrix printing

println4Ps (generic function with 1 method)

In [2]:
using Plots

backend = "gr"              #"gr" (default), "pyplot" 

if backend == "pyplot"
    pyplot(size=(600,400))
else    
    gr(size=(600,400))
end

Plots.GRBackend()

# Load Data from Text File

The following is a portion of MyData.csv:

```
date,Mkt-RF,RF,SmallGrowth
197901,4.18,0.77,10.96
197902,-3.41,0.73,-2.09
197903,5.75,0.81,11.71
197904,0.05,0.8,3.27
```

In [3]:
xx   = readdlm("Data/MyData.csv",',',header=true)  #reading the csv file    
x    = xx[1]                                #xx[1] contains the data

println("Column headers: ",xx[2])           #xx[2] contains the headers
println("\nfirst four lines of x:") 
printmat(x[1:4,:])

Column headers: AbstractString["date" "Mkt-RF" "RF" "SmallGrowth"]

first four lines of x:
197901.000     4.180     0.770    10.960
197902.000    -3.410     0.730    -2.090
197903.000     5.750     0.810    11.710
197904.000     0.050     0.800     3.270



### Creating Variables

In [4]:
ym  = x[:,1]                    #yearmonth, like 200712
Rme = x[:,2]                    #picking out the second column
Rf  = x[:,3]                   
R   = x[:,4] +                  # + 0.0 to illustrate that commands continue on the next line
      0.0                     
Re  = R - Rf                    #do R .- Rf if R has several columns (and Rf only one)

println("first 4 obs of Rme and Re")
printmat([Rme[1:4,:] Re[1:4,:]])

first 4 obs of Rme and Re
     4.180    10.190
    -3.410    -2.820
     5.750    10.900
     0.050     2.470



# Some Descriptive Statistics

## Means and Standard Deviations

The next few cells estimate means, standard deviations, covarances and correlations of the variables Rme (US equity market excess return) and Re (excess returns for a segment of the market, small growth firms). 

In [5]:
μ = mean([Rme Re],1)    #,1 to calculate average along a column, gives a row vector
σ = std([Rme Re],1)     #do \sigma[Tab] to get σ

println("            Rme       Re")
printlnPs("means: ",μ)  #for more stat functions, see the package StatsBase.jl
printlnPs("std:   ",σ)

            Rme       Re
means:      0.602     0.303
std:        4.604     8.572


## Covariances and Correlations

In [6]:
println("\n","cov([Rme Re]): ")          
printmat(cov([Rme Re]))

println("\n","cor([Rme Re]): ")          
printmat(cor([Rme Re]))


cov([Rme Re]): 
    21.197    28.426
    28.426    73.475


cor([Rme Re]): 
     1.000     0.720
     0.720     1.000



## OLS

A simple linear regression
$
y = b x + u
$,
where $x=[1,R^e_m]$.

Clearly, the first element of b is the intercept and the second element is the slope coefficient.

The GLM package (not used here) has powerful regression methods. See https://github.com/JuliaStats/GLM.jl.

In [7]:
c   = ones(size(Rme,1))         #a vector with ones, no. rows from variable, here ones(Rme) works too
x   = [c Rme]                   #x is a Tx2 matrix
y   = copy(Re)                  #to get standard OLS notation, copy to get an independent copy

b   = inv(x'x)*x'y              #OLS according to a textbook
b2  = x\y                       #also OLS, quicker and numerically more stable
u   = y - x*b                   #OLS residuals
R2a = 1 - var(u)/var(y)         #R2, but that name is already taken

println("OLS coefficients, regressing Re on constant and Rme, different calculations")
printmat([b b2])                
printlnPs("R2: ",R2a) 
printlnPs("no. of observations: ",size(Re,1))

OLS coefficients, regressing Re on constant and Rme, different calculations
    -0.504    -0.504
     1.341     1.341

R2:      0.519
no. of observations:        388


# Drawing Random Numbers and Finding Critical Values

## Random Numbers: Independent Variables

In [8]:
T = 100
x = randn(T,2)    #T x 2 matrix, N(0,1) distribution

println("\n","mean and std of random draws: ")
mu    = mean(x,1)                 
sigma = std(x,1)
printmat([mu;sigma])

println("covariance and correlation matrices:")
printmat(cov(x))
printmat(cor(x))


mean and std of random draws: 
     0.069     0.117
     1.047     1.082

covariance and correlation matrices:
     1.095     0.133
     0.133     1.170

     1.000     0.117
     0.117     1.000



## Random Numbers: Correlated Variables

In [9]:
μ = [-1;10]
Σ = [1 0.5;
     0.5 2]

d = MvNormal(μ,Σ)       

T = 100
x = rand(d,T)'          #T x 2 matrix

println("\n","mean and std of random draws: ")
mu    = mean(x,1)                 
sigma = std(x,1)
printmat([mu;sigma])

println("covariance and correlation matrices:")
printmat(cov(x))
printmat(cor(x))


mean and std of random draws: 
    -0.926    10.099
     0.986     1.395

covariance and correlation matrices:
     0.972     0.567
     0.567     1.945

     1.000     0.412
     0.412     1.000



## Quantiles ("critical values") of Distributions

In [10]:
N05     = quantile(Normal(0,1),0.05)            #from the Distributions package
Chisq05 = quantile(Chisq(5),0.95)

println("\n","5th percentile of N(0,1) and 95th of Chisquare(5)")      #lots of statistics functions
printmat([N05 Chisq05])


5th percentile of N(0,1) and 95th of Chisquare(5)
    -1.645    11.070



# Statistical Plots

In [11]:
YearFrac = floor.(ym/100) + (mod.(ym,100)-1)/12    #year + (month-1)/12, simple but works

plot3a = plot(YearFrac,Rme,color=:blue,legend=false)
plot!(xlims=(1978,2012),ylims=(-25,25))
title!("Time series plot: monthly US equity market excess return")
ylabel!("%")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1980 
 
 
 1990 
 
 
 2000 
 
 
 2010 
 
 
 -20 
 
 
 -10 
 
 
 0 
 
 
 10 
 
 
 20 
 
 
 Time series plot: monthly US equity market excess return 
 
 
 % 
 
<polyline clip-path="url(#clip02)" style="stroke:#0000ff; stroke-width:1; stroke-opacity:1; fill:none" points="
 55.6276,158.04 56.9824,212.784 58.3372,146.717 59.692,187.828 61.0468,203.912 62.4016,160.204 63.7564,182.924 65.1112,147.077 66.466,193.166 67.8208,246.9 
 69.1755,149.457 70.5303,174.701 71.8851,146.644 73.2399,193.887 74.5947,283.612 75.9495,159.555 77.3043,150.683 78.6591,165.397 80.0139,141.956 81.3687,175.783 
 82.7235,172.321 84.0783,180.544 85.4331,119.381 86.7879,222.449 88.1427,224.613 89.4975,184.727 90.8522,163.594 92.207,204.129 93.5618,186.674 94.9166,205.283 
 96.2714,199.369 97.6262,238.028 98.981,243.149 100.336,153.496 101.691,162.873 103.045,214.731 104.4,212.856 105.755,231.681 107.11,202.542 108.465,165.109 
 109.819,216.174 111.174,212.351 112.529,210.548 113.884,107.841 115.239,179.75 116.593,106.903 117.948,155.3 119.303,182.563 120.658,162.945 122.013,170.879 
 123.367,167.705 124.722,139.792 126.077,183.645 127.432,165.758 128.786,216.318 130.141,191.146 131.496,182.058 132.851,213.866 134.206,171.889 135.56,201.027 
 136.915,203.047 138.27,221.511 139.625,183.789 140.98,192.228 142.334,231.537 143.689,176.721 145.044,208.961 146.399,112.889 147.754,194.103 149.108,195.474 
 150.463,201.172 151.818,175.711 153.173,131.065 154.528,180.183 155.882,193.887 157.237,194.969 158.592,152.703 159.947,179.822 161.302,192.877 162.656,195.618 
 164.011,221.223 165.366,160.853 166.721,142.677 168.075,161.791 169.43,185.16 170.785,139.72 172.14,153.641 173.495,197.637 174.849,155.083 176.204,181.698 
 177.559,234.999 178.914,143.759 180.269,248.414 181.623,155.949 182.978,180.111 184.333,210.764 185.688,98.5364 187.043,156.742 188.397,174.485 189.752,203.624 
 191.107,187.251 192.462,160.132 193.817,159.627 195.171,164.82 196.526,206.437 197.881,355.089 199.236,242.86 200.591,140.297 201.945,157.896 203.3,154.218 
 204.655,203.335 206.01,183.573 207.365,191.579 208.719,154.578 210.074,197.133 211.429,212.64 212.784,165.83 214.138,179.894 215.493,204.129 216.848,177.514 
 218.203,144.481 219.558,204.417 220.912,177.514 222.267,158.257 223.622,165.541 224.977,196.844 226.332,137.629 227.686,177.586 229.041,193.959 230.396,214.226 
 231.751,180.327 233.106,179.39 234.46,242.86 235.815,181.553 237.17,175.423 238.525,213.577 239.88,128.974 241.234,195.762 242.589,199.873 243.944,259.233 
 245.299,231.32 246.654,202.109 248.008,144.913 249.363,171.239 250.718,156.526 252.073,136.98 253.428,170.518 254.782,189.631 256.137,162.224 257.492,222.954 
 258.847,157.968 260.201,172.177 261.556,199.441 262.911,178.38 264.266,217.905 265.621,113.899 266.975,191.507 268.33,180.544 269.685,207.735 271.04,180.832 
 272.395,185.592 273.749,204.417 275.104,161.647 276.459,205.066 277.814,181.121 279.169,181.914 280.523,160.853 281.878,177.37 283.233,180.76 284.588,185.881 
 285.943,171.889 287.297,208.24 288.652,168.426 290.007,186.097 291.362,190.497 292.717,161.502 294.071,189.631 295.426,176.721 296.781,202.686 298.136,175.783 
 299.491,167.272 300.845,207.158 302.2,223.17 303.555,183.284 304.91,183.717 306.264,210.548 307.619,168.138 308.974,160.132 310.329,204.129 311.684,180.471 
 313.038,217.689 314.393,182.275 315.748,176.505 317.103,162.512 318.458,172.033 319.812,173.331 321.167,167.561 322.522,169.076 323.877,162.007 325.232,184.871 
 326.586,165.037 327.941,199.729 329.296,160.42 330.651,180.76 332.006,171.023 333.36,179.245 334.715,183.14 336.07,173.115 337.425,171.889 338.78,197.06 
 340.134,230.238 341.489,167.705 342.844,153.136 344.199,181.337 345.553,143.831 346.908,199.729 348.263,152.847 349.618,191.795 350.973,223.675 352.327,160.709 
 353.682,140.081 355.037,159.05 356.392,136.114 357.747,217.328 359.101,149.169 360.456,216.03 361.

In [12]:
plot3b = scatter(Rme,Re,color=:blue,legend=false)
plot!([-40;60],[-40;60],color=:black)
plot!(xlims=(-40,40),ylims=(-40,60))
title!("Scatter plot: two monthly return series (and 45 degree line)")
xlabel!("Market excess return, %")
ylabel!("Excess returns on small growth stocks, %")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -40 
 
 
 -20 
 
 
 0 
 
 
 20 
 
 
 40 
 
 
 -40 
 
 
 -20 
 
 
 0 
 
 
 20 
 
 
 40 
 
 
 60 
 
 
 Scatter plot: two monthly return series (and 45 degree line) 
 
 
 Market excess return, % 
 
 
 Excess returns on small growth stocks, %

In [13]:
xGrid = -25:0.1:15
pdfX   = pdf(Normal(mean(Rme),std(Rme)),xGrid) #the N(μ,σ) pdf
                                        #"Distributions" wants σ, not σ^2

histogram(Rme,bins = -25:1:15,normalized=true,label="histogram")
plot!(xGrid,pdfX,linewidth=3,label="fitted N()")
title!("Histogram: monthly US equity market excess return")
xlabel!("Market excess return, %")
ylabel!("Number of months")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -20 
 
 
 -10 
 
 
 0 
 
 
 10 
 
 
 0.00 
 
 
 0.02 
 
 
 0.04 
 
 
 0.06 
 
 
 0.08 
 
 
 0.10 
 
 
 Histogram: monthly US equity market excess return 
 
 
 Market excess return, % 
 
 
 Number of months 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<polyline clip-path="url(#clip02)" style="stroke:#e26f46; stroke-width:3; stroke-opacity:1; fill:none" points="
 55.0141,368.504 56.3178,368.504 57.6215,368.504 58.9251,368.504 60.2288,368.504 61.5325,368.504 62.8361,368.504 64.1398,368.504 65.4435,368.504 66.7471,368.504 
 68.0508,368.504 69.3545,368.504 70.6581,368.504 71.9618,368.504 73.2655,368.504 74.5692,368.504 75.8728,368.504 77.1765,368.504 78.4802,368.503 79.7838,368.503 
 81.0875,368.503 82.3912,368.503 83.6948,368.503 84.9985,368.503 86.3022,368.503 87.6059,368.503 88.9095,368.503 90.2132,368.503 91.5169,368.503 92.8205,368.502 
 94.1242,368.502 95.4279,368.502 96.7315,368.502 98.0352,368.502 99.3389,368.501 100.643,368.501 101.946,368.501 103.25,368.5 104.554,368.5 105.857,368.5 
 107.161,368.499 108.465,368.499 109.768,368.498 111.072,368.497 112.376,368.497 113.679,368.496 114.983,368.495 116.287,368.494 117.59,368.493 118.894,368.492 
 120.198,368.491 121.501,368.489 122.805,368.488 124.109,368.486 125.412,368.484 126.716,368.482 128.02,368.48 129.323,368.478 130.627,368.475 131.931,368.473 
 133.234,368.47 134.538,368.466 135.842,368.463 137.145,368.459 138.449,368.455 139.753,368.45 141.056,368.445 142.36,368.439 143.664,368.433 144.967,368.427 
 146.271,368.42 147.575,368.412 148.878,368.404 150.182,368.394 151.486,368.385 152.789,368.374 154.093,368.362 155.397,368.35 156.7,368.336 158.004,368.322 
 159.308,368.306 160.611,368.289 161.915,368.27 163.219,368.25 164.522,368.229 165.826,368.206 167.13,368.181 168.433,368.154 169.737,368.125 171.041,368.094 
 172.344,368.06 173.648,368.024 174.952,367.985 176.255,367.944 177.559,367.899 178.863,367.851 180.166,367.8 181.47,367.745 182.774,367.686 184.077,367.623 
 185.381,367.555 186.685,367.483 187.988,367.406 189.292,367.323 190.596,367.235 191.899,367.141 193.203,367.041 194.507,366.934 195.81,366.82 197.114,366.698 
 198.418,366.569 199.721,366.432 201.025,366.285 202.329,366.13 203.632,365.965 204.936,365.79 206.24,365.604 207.543,365.406 208.847,365.197 210.151,364.975 
 211.454,364.741 212.758,364.492 214.062,364.229 215.365,363.951 216.669,363.658 217.973,363.347 219.276,363.02 220.58,362.674 221.884,362.31 223.187,361.926 
 224.491,361.521 225.795,361.095 227.098,360.647 228.402,360.176 229.706,359.68 231.01,359.16 232.313,358.613 233.617,358.039 234.921,357.437 236.224,356.806 
 237.528,356.145 238.832,355.453 240.135,354.729 241.439,353.971 242.743,353.179 244.046,352.351 245.35,351.486 246.654,350.584 247.957,349.643 249.261,348.661 
 250.565,347.639 251.868,346.574 253.172,345.465 254.476,344.312 255.779,343.113 257.083,341.868 258.387,340.574 259.69,339.232 260.994,337.839 262.298,336.395 
 263.601,334.899 264.905,333.35 266.209,331.747 267.512,330.089 268.816,328.376 270.12,326.605 271.423,324.777 272.727,322.891 274.031,320.946 275.334,318.941 
 276.638,316.876 277.942,314.751 279.245,312.564 280.549,310.316 281.853,308.006 283.156,305.634 284.46,303.2 285.764,300.703 287.067,298.145 288.371,295.524 
 289.675,292.842 290.978,290.097 292.282,287.292 293.586,284.426 294.889,281.499 296.193,278.514 297.497,275.47 298.8,272.368 300.104,269.209 301.408,265.996 
 302.711,262.728 304.015,259.407 305.319,256.036 306.622,252.614 307.926,249.145 309.23,245.63 310.533,242.072 311.837,238.471 313.141,234.831 314.444,231.155 
 315.748,227.443 317.052,223.7 318.355,219.927 319.659,216.128 320.963,212.305 322.266,208.462 323.57,204.602 324.874,200.728 326.177,196.844 327.481,192.952 
 328.785,189.057 330.088,185.161 331.392,181.

In [16]:
println("\n","end of program")


end of program
